In [1]:
import numpy as np
import pandas as pd

In [2]:
def import_excel(start_year, end_year):
    df = pd.read_excel('./data/migr/migr_combined_2007-2019.xlsx', sheet_name=[str(i) for i in range(start_year, end_year+1)])  
    df = pd.concat(df, axis=0, ignore_index=True)
    return df

### 2015 - 2019

In [3]:
migr_2015_2019 = import_excel(2015, 2019)

In [4]:
migr_2015_2019['Planungsraum'] = migr_2015_2019['Planungsraum'].astype(int).apply('{:0>8}'.format)
migr_2015_2019['LOR'] = migr_2015_2019['Planungsraum'].apply(lambda x: str(x)[:6])
migr_2015_2019['Bezirk'] = migr_2015_2019['LOR'].apply(lambda x: str(x)[:2])
migr_2015_2019.drop('Planungsraum', axis=1, inplace=True)


migr_2015_2019.columns = ['total_foreigners', 'EU', 'FRA', 'GRC', 'ITA', 'AUT', 'ESP',
                          'UK', 'POL', 'BGR', 'ROU', 'HRV', 'former_JUGO', 'BIH', 'SRB',
                          'former_UDSSR', 'RUS', 'UKR', 'KAZ', 'Islamic_countries', 'TUR', 'IRN',
                          'Arabic_countries', 'LBN', 'SYR', 'VNM', 'USA', 'not_identified',
                          'year', 'LOR', 'Bezirk']


In [5]:
migr_2015_2019_clean = migr_2015_2019.groupby(['Bezirk', 'LOR', 'year'], as_index=False).sum()
migr_2015_2019_clean.sort_values(['year', 'LOR'], inplace=True, ignore_index=True)

### 2009 - 2014

In [6]:
migr_2009_2014 = import_excel(2009, 2014)

In [7]:
migr_2009_2014['Bezirk'] = migr_2009_2014['Bezirk'].apply('{:0>2}'.format)
migr_2009_2014.drop('Bezirksname', axis=1, inplace=True)

migr_2009_2014.columns = ['Bezirk', 'total_foreigners', 'EU', 'POL', 'ITA',
                          'former_UDSSR', 'RUS', 'UKR',
                          'KAZ', 'former_JUGO', 'TUR', 'Arabic_countries',
                          'VNM', 'USA', 'year', 'FRA',
                          'GRC', 'AUT', 'UK', 'BGR',
                          'BIH', 'HRV', 'SRB', 'Islamic_countries',
                          'IRN', 'LBN', 'not_identified', 'ESP', 'ROU',
                          'CHN']

### 2007 - 2008

In [8]:
migr_2007_2008 = import_excel(2007, 2008)

In [9]:
migr_2007_2008['Bezirk'] = migr_2007_2008['Bezirk'].apply('{:0>2}'.format)
migr_2007_2008.drop(['Ortsteil', 'Ortsteilname'], axis=1, inplace=True)

migr_2007_2008.columns = ['Bezirk', 'total_foreigners', 'year']

In [10]:
migr_2007_2008_clean = migr_2007_2008.groupby(['Bezirk', 'year'], as_index=False).sum()
migr_2007_2008_clean.sort_values(['year', 'Bezirk'], inplace=True, ignore_index=True)

### Expanding 2007-2008 and 2009-2014 to LOR-level

In [11]:
all_lor = migr_2015_2019_clean[['Bezirk', 'LOR']].copy().drop_duplicates()

In [12]:
migr_2007_2008_lor = pd.merge(all_lor, migr_2007_2008_clean, on='Bezirk', how='outer')
migr_2009_2014_lor = pd.merge(all_lor, migr_2009_2014, on='Bezirk', how='outer')

### Dividing totals for 2007-20014 by number of LOR in Bezirk

##### 2007 - 2008

In [13]:
#group_2007_2008 = migr_2007_2008_lor.groupby(['Bezirk', 'year']).agg(['count', 'mean'])

In [14]:
#group_2007_2008['total_foreigners_new'] = group_2007_2008[('total_foreigners', 'mean')] / group_2007_2008[('total_foreigners', 'count')]
#group_2007_2008.drop([('total_foreigners', 'count'), ('total_foreigners', 'mean')], axis=1, inplace=True)
#group_2007_2008.reset_index(inplace=True)
#group_2007_2008.columns = ['Bezirk', 'year', 'total_foreigners']

##### 2009 - 2014

In [15]:
#group_2009_2014 = migr_2009_2014_lor.groupby(['Bezirk', 'year']).agg(['count', 'mean'])

In [16]:
#cols = ['total_foreigners', 'EU', 'POL', 'ITA', 'former_UDSSR',
#       'RUS', 'UKR', 'KAZ', 'former_JUGO', 'TUR', 'Arabic_countries', 'VNM',
#       'USA', 'FRA', 'GRC', 'AUT', 'UK', 'BGR', 'BIH', 'HRV', 'SRB',
#       'Islamic_countries', 'IRN', 'LBN', 'not_identified', 'ESP', 'ROU',
#       'CHN']

#for col in cols:
#    group_2009_2014['_'.join([col, 'new'])] = group_2009_2014[(col, 'mean')] / group_2009_2014[(col, 'count')]
#    group_2009_2014.drop([(col, 'count'), (col, 'mean')], axis=1, inplace=True)

In [17]:
#group_2009_2014.reset_index(inplace=True)

#cols_new = ['Bezirk', 'year'] + ['_'.join([c, 'new']) for c in cols]
    
#group_2009_2014.columns = cols_new

In [18]:
#migr_2009_2014_split = pd.merge(migr_2009_2014_lor, group_2009_2014, on=['Bezirk', 'year'], how='outer')
#migr_2009_2014_split.drop(cols, axis=1, inplace=True)
#migr_2009_2014_split.columns = ['Bezirk', 'LOR', 'year'] + cols

### Combining dataframes

In [19]:
migr = migr_2015_2019_clean.append([migr_2009_2014_lor, migr_2007_2008_lor], ignore_index=True)

In [22]:
import joblib

joblib.dump(migr, './dataframes/final_migr.pkl')

['./dataframes/final_migr.pkl']

In [23]:
migr[migr['year']==2014]

,Bezirk,LOR,year,total_foreigners,EU,FRA,GRC,ITA,AUT,ESP,...,Islamic_countries,TUR,IRN,Arabic_countries,LBN,SYR,VNM,USA,not_identified,CHN
695,01,010111,2014,171041,53080.0,4044.0,2982.0,4862.0,2068.0,3050.0,...,63518.0,36432.0,908.0,18083.0,6012.0,NaN,2284.0,3095.0,12126.0,2207.0
701,01,010112,2014,171041,53080.0,4044.0,2982.0,4862.0,2068.0,3050.0,...,63518.0,36432.0,908.0,18083.0,6012.0,NaN,2284.0,3095.0,12126.0,2207.0
707,01,010113,2014,171041,53080.0,4044.0,2982.0,4862.0,2068.0,3050.0,...,63518.0,36432.0,908.0,18083.0,6012.0,NaN,2284.0,3095.0,12126.0,2207.0
713,01,010114,2014,171041,53080.0,4044.0,2982.0,4862.0,2068.0,3050.0,...,63518.0,36432.0,908.0,18083.0,6012.0,NaN,2284.0,3095.0,12126.0,2207.0
719,01,010221,2014,171041,53080.0,4044.0,2982.0,4862.0,2068.0,3050.0,...,63518.0,36432.0,908.0,18083.0,6012.0,NaN,2284.0,3095.0,12126.0,2207.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,12,122311,2014,69265,22149.0,1174.0,894.0,1743.0,606.0,460.0,...,24975.0,14664.0,490.0,5599.0,1572.0,NaN,823.0,550.0,6369.0,432.0
1499,12,123012,2014,69265,22149.0,1174.0,894.0,1743.0,606.0,460.0,...,24975.0,14664.0,490.0,5599.0,1572.0,NaN,823.0,550.0,6369.0,432.0
1505,12,123021,2014,69265,22149.0,1174.0,894.0,1743.0,606.0,460.0,...,24975.0,14664.0,490.0,5599.0,1572.0,NaN,823.0,550.0,6369.0,432.0
1511,12,123022,2014,69265,22149.0,1174.0,894.0,1743.0,606.0,460.0,...,24975.0,14664.0,490.0,5599.0,1572.0,NaN,823.0,550.0,6369.0,432.0
